# Loss Analysis

In [ ]:
import pandas as pd

df = pd.read_parquet('fixtypes.parquet')
for i in df.columns: print(f"[{i}]", end = ", ")

[Row ID], [Order ID], [Order Date], [Ship Date], [Ship Mode], [Customer ID], [Customer Name], [Segment], [Country], [City], [State], [Postal Code], [Region], [Product ID], [Category], [Sub-Category], [Product Name], [Sales], [Quantity], [Discount], [Profit], 

### Find what is likely to indicate a loss / less profit

In [3]:
lists = ["Ship Mode", "Segment", "Country", "Region", "Category", "Discount"]

for i in lists:
    data = df[[i, "Profit"]]
    data = data.groupby([i]).sum()
    print(data)

                     Profit
Ship Mode                  
First Class      48969.8399
Same Day         15891.7589
Second Class     57446.6354
Standard Class  164088.7875
                  Profit
Segment                 
Consumer     134119.2092
Corporate     91979.1340
Home Office   60298.6785
                    Profit
Country                   
United States  286397.0217
              Profit
Region              
Central   39706.3625
East      91522.7800
South     46749.4303
West     108418.4489
                      Profit
Category                    
Furniture         18451.2728
Office Supplies  122490.8008
Technology       145454.9481
               Profit
Discount             
0.00      320987.6032
0.10        9029.1770
0.15        1418.9915
0.20       90337.3060
0.30      -10369.2774
0.32       -2391.1377
0.40      -23057.0504
0.45       -2493.1111
0.50      -20506.4281
0.60       -5944.6552
0.70      -40075.3569
0.80      -30539.0392


from finding total profit from each discount determine over discount make less profit

## What is point of Discount?

The purpose of discount that make loss
- Loss Leader Strategy
- Urgent Clearance
- Market Penetration

from data it can analysis Loss Leader Strategy

## Loss Leader Strategy

Using a loss leader strategy, a store sells a discounted product at a loss to lure customers in, hoping they will also buy additional that allowing the store to make profit

### Chain of Thought
- when customer get the product that make loss the next product must reduce the loss or make the profit to the store
- determine in graph between number of order of customer after buy products that make loss from discount and sum of profit from each customer should be Positive Correlation

### Analytical Approach
**hypothesis:** when sell an heavy discount product the strategic expectation is that the profit from this customer's subsequent purchases will leading to a profitable long-term relationship

analyzed the data to visualize the relationship between two key
**X-axis: Total Number of Orders**
**Y-axis: Total Net Profit**

### Expected Outcome
if the hypothesis is correct, the resulting should be positive corelation between x-axis and y-axis


### data preparation

In [121]:
import matplotlib.pyplot as plt

customers = dict()

#sorted data from Order date
sorted_data = df.sort_values("Order Date")

for idx,row in df.iterrows():
    id, profit = row["Customer ID"], row["Profit"]

    if id not in customers and profit < 0:
        customers[id] = [profit]
        continue
    
    if id in customers:
        customers[id].append(customers[id][len(customers[id])-1] + profit)
    

### Spearman's Rank Correlation

In [ ]:
from scipy.stats import spearmanr

for id in customers:
    

    break

ModuleNotFoundError: No module named 'scipy'